In [2]:
import datetime as dt
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
windows = r'C:\Users\Usuario\Downloads\effis_layer\modis.ba.poly.shp'
linux = '~/Descargas/effis_layer/modis.ba.poly.shp'

effis = gpd.read_file()
effis.head()

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,SCLEROPH,TRANSIT,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry
0,2,2016-08-28 00:00:00,2022-01-26 11:57:54.973474,AL,Korcë,Konispol,67,0,0,0,19.402985074597908,16.417910447736688,10.447761194014257,53.731343283501886,0,0,0,FireSeason,"POLYGON ((20.18901 39.74908, 20.18909 39.75006..."
1,51,2016-05-06 00:00:00,2022-01-26 11:57:54.973474,TR,Mardin,N.A.,171,51.724137931004755,0,0,0,42.52873563215947,5.747126436778307,0,0,0,0,FireSeason,"POLYGON ((41.55879 37.29378, 41.55996 37.29843..."
2,888,2016-08-08 00:00:00,2022-01-26 11:57:54.973474,PT,Área Metropolitana do Porto,Covelo de Paivó e Janarde,26593,0.6020922706404734,0.2558892150222012,0.2220215247986746,0,56.893956498833234,38.661849928501404,3.2174305712350297,0.14675999096861542,0,9.843358251326316,FireSeason,"POLYGON ((-8.06251 40.84474, -8.06033 40.8436,..."
3,865,2016-10-25 00:00:00,2022-01-26 11:57:54.973474,EL,???????,?????? ????????? ???????,165,0,0,0,1.204819277107708,0,98.79518072283206,0,0,0,0,FireSeason,"POLYGON ((24.49178 35.25119, 24.49455 35.251, ..."
4,19,2016-08-11 00:00:00,2022-01-26 11:57:54.973474,PT,Tâmega e Sousa,Caçarilhe e Infesta,81,0,1.17647058823391,0,0,67.05882352933287,19.99999999997647,11.7647058823391,0,0,0,FireSeason,"POLYGON ((-8.05657 41.40623, -8.05945 41.4048,..."


In [13]:
effis.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 73169 entries, 0 to 73168
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          73169 non-null  object  
 1   FIREDATE    73169 non-null  object  
 2   LASTUPDATE  73169 non-null  object  
 3   COUNTRY     73167 non-null  object  
 4   PROVINCE    73169 non-null  object  
 5   COMMUNE     73169 non-null  object  
 6   AREA_HA     73169 non-null  object  
 7   BROADLEA    72455 non-null  object  
 8   CONIFER     72455 non-null  object  
 9   MIXED       72455 non-null  object  
 10  SCLEROPH    72455 non-null  object  
 11  TRANSIT     72455 non-null  object  
 12  OTHERNATLC  72455 non-null  object  
 13  AGRIAREAS   72455 non-null  object  
 14  ARTIFSURF   72455 non-null  object  
 15  OTHERLC     72455 non-null  object  
 16  PERCNA2K    73169 non-null  object  
 17  CLASS       73169 non-null  object  
 18  geometry    73169 non-null  geometry
d

In [10]:
effis.AREA_HA = effis.AREA_HA.astype(int)
effis = effis[effis.AREA_HA > 499]
effis.AREA_HA.describe()

count     3018.000000
mean      1869.095427
std       3960.824142
min        500.000000
25%        651.250000
50%        934.000000
75%       1635.000000
max      96610.000000
Name: AREA_HA, dtype: float64

In [4]:
# Pasamos las columnas de inicio de incendio y última actualización a datetime 
effis.FIREDATE = pd.to_datetime(effis.FIREDATE, format='ISO8601')
effis.LASTUPDATE = pd.to_datetime(effis.LASTUPDATE, format='ISO8601')

# Eliminamos los incendios de 2025
effis = effis[effis.FIREDATE.dt.year < 2025]

# Calculamos la duración del incendio
effis['DURATION'] = effis.LASTUPDATE - effis.FIREDATE

In [ ]:
# Calculamos el día juliano pero como si todos los años fueran bisiestos

def dia_juliano_bisiesto(serie):
    dias_meses = [0,31,29,31,30,31,30,31,31,30,31,30]
    lista_dias = []
    for valor in serie:
        dia_juliano = 0
        mes = valor.month
        for index in range(mes):
            dia_juliano += dias_meses[index]
        dia_juliano += valor.day
        lista_dias.append(dia_juliano)
    return lista_dias

effis['DAY_YEAR'] = dia_juliano_bisiesto(effis.FIREDATE)

In [50]:
# Calculamos centroides y extraemos coordenadas
effis['CENTROID'] = effis.geometry.centroid.to_crs('WGS 84')
effis['LAT'], effis['LON'] = effis.CENTROID.y, effis.CENTROID.x

/tmp/ipykernel_2719/2172088228.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  effis['CENTROID'] = effis.geometry.centroid.to_crs('WGS 84')
